In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

In [ ]:
train.head()

In [ ]:
print(train.isnull().sum())
print(test.isnull().sum())


In [ ]:
train.drop(['id'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

In [ ]:
sns.distplot(train['target'])
sns.despine(trim=True, left=True)

In [ ]:
features = [f'cont{x}'for x in range(1,15)]
features

In [ ]:
i = 1
plt.figure()
fig, ax = plt.subplots(5, 3,figsize=(14, 24))
for feature in features:
    plt.subplot(5, 3,i)
    sns.distplot(train[feature],color="blue", kde=True,bins=120, label='train')
    sns.distplot(test[feature],color="green", kde=True,bins=120, label='test')
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
train.corr()

In [ ]:
corr = train.corr()
plt.subplots(figsize=(14,10))
sns.heatmap(corr, vmax=0.9, cmap="viridis", square=True)

In [ ]:

for col in train.columns[:-1]:
    plt.boxplot([train[col], test[col]], labels=['train', 'test'])
    plt.title(col)
    plt.legend()
    plt.show()

In [ ]:
plt.boxplot(train['target'])

In [ ]:
Q1_train = train.quantile(0.25)
Q3_train = train.quantile(0.75)
IQR_train = Q3_train - Q1_train
((train < Q1_train - 1.5*IQR_train) | (train > Q3_train + 1.5*IQR_train)).agg([sum, 'mean'])

In [ ]:
Q1_test = test.quantile(0.25)
Q3_test = test.quantile(0.75)
IQR_test = Q3_test - Q1_test
((test < Q1_test - 1.5 * IQR_test) | (test > Q3_test + 1.5 * IQR_test)).agg([sum, 'mean'])

In [ ]:
def replace_outliers(data):
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        median_ = data[col].median()
#         data[col].mask(((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), median_, inplace=True)
        # data[col] = np.where(((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)),
        #                     median_, data[col])
        data.loc[((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), col] = median_
    return data

In [ ]:
train = replace_outliers(train)

In [ ]:
sns.distplot(train['target'])

In [ ]:
X = train.drop('target', axis=1)
y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.18, random_state=33)

In [ ]:
import lightgbm as lgb
LGB = lgb.LGBMRegressor(random_state=33, n_estimators=5000, min_data_per_group=5, boosting_type='gbdt',
 num_leaves=246, max_dept=-1, learning_rate=0.005, subsample_for_bin=200000,
 lambda_l1= 1.074622455507616e-05, lambda_l2= 2.0521330798729704e-06, n_jobs=-1, cat_smooth=1.0, 
 importance_type='split', metric='rmse', min_child_samples=20, min_gain_to_split=0.0, feature_fraction=0.5, 
 bagging_freq=6, min_sum_hessian_in_leaf=0.001, min_data_in_leaf=100, bagging_fraction=0.82063411)

LGB.fit(X_train, y_train)

In [ ]:
pred = LGB.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(y_test, pred)

In [ ]:
pred = LGB.predict(test)
pred